In [1]:
import os

import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from sklearn.preprocessing import OneHotEncoder

from helper import calculate_iqr_range, get_datatype_mapping_for_reduction, reduce_dataset_size

# set global variables
DATA_DIR = "../data/dataset"
max_std = 402198.1853209468
min_std = 0.0

# set maximum column displayable at once
pd.options.display.max_columns = 122

# load the dataset
df = pd.read_csv(os.path.join(DATA_DIR, "train_data.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "test_data.csv"))
df = reduce_dataset_size(df)
df.info()
df.head()

Reducing dataset size
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184506 entries, 0 to 184505
Columns: 122 entries, SK_ID_CURR to TARGET
dtypes: float32(51), float64(13), int16(3), int32(1), int64(1), object(16), uint32(1), uint8(36)
memory usage: 86.9+ MB


,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,Cash loans,F,N,Y,1,67500.0,227520.0,11065.5,180000.0,Family,Commercial associate,Secondary / secondary special,Married,House / apartment,0.018634,-17210,-774,-5580,-755,NaN,1,1,1,1,0,0,Sales staff,3.0,2,2,WEDNESDAY,12,0,0,0,0,0,0,Business Entity Type 3,NaN,0.264354,NaN,0.0124,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0095,NaN,0.000,0.0126,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0099,NaN,0.000,0.0125,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0096,NaN,0.0000,NaN,block of flats,0.0074,Mixed,No,0.0,0.0,0.0,0.0,-150.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,Cash loans,M,Y,Y,1,247500.0,1882372.5,65560.5,1719000.0,Unaccompanied,Working,Higher education,Civil marriage,House / apartment,0.018850,-18640,-96,-5711,-2175,25.0,1,1,0,1,0,1,Managers,3.0,2,2,TUESDAY,10,0,0,0,0,1,1,Business Entity Type 3,0.581064,0.172251,0.686382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-444.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,Cash loans,M,Y,Y,1,180000.0,101880.0,10827.0,90000.0,"Spouse, partner",Working,Higher education,Married,House / apartment,0.022625,-14649,-6130,-1776,-4568,9.0,1,1,1,1,0,0,Laborers,3.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 4,NaN,0.636218,0.540654,0.1979,0.1169,0.9841,0.7824,0.1119,0.16,0.0690,0.4583,0.5,0.1587,0.1614,0.1033,0.0,0.188,0.2017,0.1213,0.9841,0.7909,0.113,0.1611,0.0690,0.4583,0.5,0.1623,0.1763,0.1076,0.0,0.199,0.1999,0.1169,0.9841,0.7853,0.1127,0.16,0.0690,0.4583,0.5,0.1615,0.1642,0.1051,0.0,0.1919,reg oper account,block of flats,0.1789,Panel,No,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,

In [2]:
numerical_columns = list(df.select_dtypes(exclude=["object"]).columns)
print(f"Number of numerical columns: {len(numerical_columns)}\n" )

df[numerical_columns].describe()

Number of numerical columns: 106



,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
count,184506.000000,184506.000000,1.845060e+05,1.845060e+05,184500.000000,1.843390e+05,184506.000000,184506.000000,184506.000000,184506.00000,184506.000000,62873.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184505.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,80432.000000,184093.000000,147850.000000,90931.000000,76531.000000,94606.000000,61749.000000,55535.000000,86186.000000,91699.000000,92780.000000,59262.000000,74963.000000,58294.000000,91972.000000,56360.000000,82718.000000,90931.000000,76531.000000,94606.000000,61749.000000,55535.000000,86186.000000,91699.000000,92780.000000,59262.000000,74963.000000,58294.000000,91972.00000,56360.000000,82718.000000,90931.000000,76531.000000,94606.000000,61749.000000,55535.000000,86186.000000,91699.000000,92780.000000,59262.000000,74963.000000,58294.000000,91972.000000,56360.000000,82718.000000,95558.000000,183867.000000,183867.000000,183867.000000,183867.000000,184505.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.0,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,184506.000000,159508.000000,159508.000000,159508.000000,159508.000000,159508.000000,159508.000000,184506.000000
mean,278031.658488,0.416463,1.689985e+05,5.992853e+05,27114.455078,5.386926e+05,0.020868,-16036.718307,63838.583271,-4984.89506,-2994.113948,12.049115,0.999995,0.819849,0.199115,0.998043,0.281611,0.056638,2.152706,2.051787,2.031332,12.062963,0.015176,0.051299,0.041175,0.078778,0.230323,0.179257,0.502568,0.514616,0.510894,0.117193,0.088303,0.977795,0.752196,0.044500,0.078558,0.149578,0.225968,0.231827,0.066468,0.100229,0.107146,0.008890,0.028265,0.113888,0.087360,0.977083,0.759342,0.042394,0.074057,0.144987,0.221899,0.227819,0.065124,0.104988,0.10566,0.008094,0.026867,0.117568,0.087798,0.977831,0.755489,0.044459,0.077690,0.149065,0.225543,0

In [3]:
df[numerical_columns]

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
0,450407,1,67500.0,227520.0,11065.5,180000.0,0.018634,-17210,-774,-5580,-755,NaN,1,1,1,1,0,0,3.0,2,2,12,0,0,0,0,0,0,NaN,0.264354,NaN,0.0124,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0095,NaN,0.0000,0.0126,NaN,0.9444,NaN,NaN,0.0000,0.0345,0.0417,NaN,NaN,NaN,0.0099,NaN,0.000,0.0125,NaN,0.9444,NaN,NaN,0.00,0.0345,0.0417,NaN,NaN,NaN,0.0096,NaN,0.0000,0.0074,0.0,0.0,0.0,0.0,-150.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,2.0,0
1,271298,1,247500.0,1882372.5,65560.5,1719000.0,0.018850,-18640,-96,-5711,-2175,25.0,1,1,0,1,0,1,3.0,2,2,10,0,0,0,0,1,1,0.581064,0.172251,0.686382,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-444.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,3.0,0
2,122238,1,180000.0,101880.0,10827.0,90000.0,0.022625,-14649,-6130,-1776,-4568,9.0,1,1,1,1,0,0,3.0,2,2,14,0,0,0,0,0,0,NaN,0.636218,0.540654,0.1979,0.1169,0.9841,0.7824,0.1119,0.1600,0.0690,0.4583,0.5000,0.1587,0.1614,0.1033,0.0,0.1880,0.2017,0.1213,0.9841,0.7909,0.1130,0.1611,0.0690,0.4583,0.5000,0.1623,0.1763,0.1076,0.0,0.199,0.1999,0.1169,0.9841,0.7853,0.1127,0.16,0.0690,0.4583,0.5000,0.1615,0.1642,0.1051,0.0,0.1919,0.1789,1.0,0.0,1.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,0.0,1.0,0
3,305311,0,81000.0,405000.0,20677.5,405000.0,0.035792,-12762,-5891,-6430,-4618,NaN,1,1,1,1,1,0,1.0,2,2,18,0,0,0,0,1,1,NaN,0.599924,0.337673,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.0,7.0,1.0,-1671.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,0
4,414121,0,157500.0,888840.0,29506.5,675000.0,0.018801,-16329,-345,-1413,-4624,NaN,1,1,0,1,0,0,2.0,2,2,9,0,0,0,0,1,1,0.476511,0.312779,0.819318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
def col_info(column_name):
    print(f"{column_name}")
    print(f"Min value: {df[column_name].min()}")
    print(f"Max value: {df[column_name].max()}")
    print(f"Std: {df[column_name].std()}")
    print(f"Min std: {min_std}")
    print(f"Max std:  {max_std}\n")
    
    print(f"Pearson's correlation: {df[column_name].corr(df['TARGET'])}")
    print(f"Spearman's correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
    print(f"Missing value(s): {df[column_name].isna().sum()}")

    print("\n")
    iqr_range, lower_bound, upper_bound = calculate_iqr_range(
                df[column_name],
                scaled_factor=1.7,
                percentile_range=(25, 75)
            )
    print(f"Lower bound: {lower_bound}")
    print(f"Upper bound: {upper_bound}")
    index_of_outliers = df[
                (df[column_name] > upper_bound) | (df[column_name] < lower_bound)].index
    column_index = df[column_name].index
    column_index = column_index.drop(index_of_outliers)
    column_mean_without_outliers = df[column_name][column_index].mean()
    print(f"Number of outlier(s): {len(index_of_outliers)}")    
    mean_of_column = df[column_name].mean()
    print(f"Mean with outliers: {mean_of_column}")
    print(f"Mean without outliers: {column_mean_without_outliers}")
    more_than_mean = list()
    less_than_mean = list()
    for index, value in df[column_name][index_of_outliers].iteritems():
        if value > mean_of_column:
            if value not in more_than_mean:
                more_than_mean.append(value)
        elif value < mean_of_column:
            if value not in less_than_mean:
                less_than_mean.append(value)

            
    less_than_mean.sort()
    more_than_mean.sort()
    print(f"Less than mean: {len(less_than_mean)}")
    print(f"More than mean: {len(more_than_mean)}\n")
    
    unique_values = np.sort(df[column_name].unique())
    print(f"Unique values: {len(unique_values)}")
#     for unique_value in unique_values:
#         print(unique_value, end=", ")

#     print("\n")
#     print(df[column_name].value_counts())
    
    return index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers

# CNT_CHILDREN

In [5]:
column_name = "CNT_CHILDREN"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

CNT_CHILDREN
Min value: 0
Max value: 19
Std: 0.7196143256712833
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.020010553326490807
Spearman's correlation: 0.020319501696053968
Missing value(s): 0


Lower bound: -1.7
Upper bound: 2.7
Number of outlier(s): 2502
Mean with outliers: 0.4164634212437536
Mean without outliers: 0.37834333311355794
Less than mean: 0
More than mean: 11

Unique values: 14


In [6]:
print("Outliers:")
df[column_name][index_of_outliers]

Outliers:


78        3
269       3
379       3
406       3
469       3
         ..
184351    3
184353    3
184376    3
184407    3
184466    3
Name: CNT_CHILDREN, Length: 2502, dtype: uint8

In [7]:
df[column_name].replace([3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], 3, inplace=True)
df[column_name].value_counts()

0    129082
1     36984
2     15938
3      2502
Name: CNT_CHILDREN, dtype: int64

In [8]:
df.shape

(184506, 122)

# AMT_INCOME_TOTAL

In [9]:
column_name = "AMT_INCOME_TOTAL"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

AMT_INCOME_TOTAL
Min value: 26100.0
Max value: 117000000.0
Std: 294245.3125
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.0006852676061457032
Spearman's correlation: -0.01961855941416481
Missing value(s): 0


Lower bound: -40500.0
Upper bound: 355500.0
Number of outlier(s): 8154
Mean with outliers: 168998.5
Mean without outliers: 154557.671875
Less than mean: 0
More than mean: 220

Unique values: 1768


In [10]:
print("Outliers:")
df[column_name][index_of_outliers]

Outliers:


71        630000.0
179       382500.0
195       382500.0
240       360000.0
248       540000.0
            ...   
184349    360000.0
184358    427500.0
184383    450000.0
184387    360000.0
184483    450000.0
Name: AMT_INCOME_TOTAL, Length: 8154, dtype: float32

In [11]:
more_than_mean

[356625.0,
 356850.0,
 357750.0,
 358200.0,
 359100.0,
 360000.0,
 360450.0,
 360508.5,
 362250.0,
 362911.5,
 364500.0,
 364891.5,
 365850.0,
 366750.0,
 369000.0,
 371250.0,
 371392.1875,
 371925.0,
 373500.0,
 373950.0,
 374220.0,
 375750.0,
 376875.0,
 377145.0,
 378000.0,
 378900.0,
 380700.0,
 381150.0,
 381375.0,
 382500.0,
 383400.0,
 385200.0,
 386473.5,
 387000.0,
 388350.0,
 389250.0,
 391500.0,
 391972.5,
 392625.0,
 392850.0,
 394353.0,
 395100.0,
 396000.0,
 396922.5,
 400500.0,
 402750.0,
 403200.0,
 403650.0,
 405000.0,
 409500.0,
 410400.0,
 410850.0,
 411300.0,
 411750.0,
 414000.0,
 415813.5,
 416029.5,
 416322.0,
 418500.0,
 422100.0,
 423000.0,
 424350.0,
 426019.5,
 427050.0,
 427500.0,
 430650.0,
 432000.0,
 432981.0,
 433350.0,
 433800.0,
 434250.0,
 436495.5,
 436500.0,
 438750.0,
 440100.0,
 441000.0,
 443250.0,
 443700.0,
 445500.0,
 450000.0,
 450225.0,
 452250.0,
 454500.0,
 458550.0,
 459000.0,
 460417.5,
 463500.0,
 466956.0,
 468000.0,
 469800.0,
 472500

In [12]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] >= 9000000)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184506
Rows after: 184502


In [13]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 8154
After deleting: 8150


In [14]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 356625.0
Index of lowest outlier value in unique values: 1548
Outliers will be trimmed to: 355500.0


In [15]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

71        355500.0
179       355500.0
195       355500.0
240       355500.0
248       355500.0
            ...   
184349    355500.0
184358    355500.0
184383    355500.0
184387    355500.0
184483    355500.0
Name: AMT_INCOME_TOTAL, Length: 8150, dtype: float32

In [16]:
df.shape

(184502, 122)

# AMT_CREDIT

In [17]:
column_name = "AMT_CREDIT"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

AMT_CREDIT
Min value: 45000.0
Max value: 4050000.0
Std: 402193.53125
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.03064610970155867
Spearman's correlation: -0.017773260352640707
Missing value(s): 0


Lower bound: -645705.0
Upper bound: 1724355.0
Number of outlier(s): 3181
Mean with outliers: 599276.3125
Mean without outliers: 574997.1875
Less than mean: 0
More than mean: 387

Unique values: 4854


In [18]:
more_than_mean

[1724688.0,
 1725120.0,
 1726866.0,
 1728000.0,
 1729615.5,
 1730511.0,
 1730655.0,
 1731901.5,
 1732500.0,
 1734543.0,
 1735474.5,
 1736937.0,
 1737000.0,
 1739470.5,
 1740294.0,
 1741077.0,
 1741293.0,
 1741500.0,
 1741972.5,
 1743399.0,
 1744398.0,
 1745113.5,
 1746000.0,
 1746288.0,
 1746684.0,
 1747008.0,
 1748835.0,
 1749325.5,
 1750500.0,
 1752039.0,
 1755000.0,
 1756714.5,
 1757074.5,
 1759180.5,
 1759500.0,
 1759576.5,
 1762110.0,
 1764000.0,
 1764108.0,
 1767145.5,
 1768248.0,
 1768500.0,
 1769035.5,
 1769220.0,
 1772181.0,
 1772352.0,
 1772406.0,
 1773000.0,
 1773963.0,
 1774039.5,
 1777212.0,
 1777500.0,
 1778890.5,
 1779030.0,
 1782000.0,
 1782247.5,
 1783818.0,
 1786500.0,
 1787283.0,
 1788502.5,
 1788750.0,
 1791000.0,
 1792318.5,
 1793677.5,
 1795203.0,
 1795500.0,
 1795972.5,
 1797354.0,
 1798141.5,
 1798416.0,
 1798605.0,
 1800000.0,
 1802385.0,
 1803532.5,
 1804500.0,
 1805922.0,
 1805985.0,
 1807420.5,
 1808842.5,
 1809000.0,
 1812456.0,
 1813500.0,
 1817428.5,
 181

In [19]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 1724688.0
Index of lowest outlier value in unique values: 4467
Outliers will be trimmed to: 1724220.0


In [20]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

1         1724220.0
68        1724220.0
71        1724220.0
116       1724220.0
152       1724220.0
            ...    
184126    1724220.0
184131    1724220.0
184261    1724220.0
184387    1724220.0
184495    1724220.0
Name: AMT_CREDIT, Length: 3181, dtype: float32

In [21]:
df.shape

(184502, 122)

# AMT_ANNUITY

In [22]:
column_name = "AMT_ANNUITY"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

AMT_ANNUITY
Min value: 1615.5
Max value: 258025.5
Std: 14464.37109375
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.011574662477110031
Spearman's correlation: 0.0016367926821796897
Missing value(s): 6


Lower bound: -14113.349999999999
Upper bound: 65264.85
Number of outlier(s): 3191
Mean with outliers: 27113.099609375
Mean without outliers: 26200.21484375
Less than mean: 0
More than mean: 1201

Unique values: 12411


In [23]:
test_df[column_name].isna().sum()

6

In [24]:
test_df[column_name]

0         25317.0
1         21069.0
2         36553.5
3         20682.0
4         31018.5
           ...   
123000    32602.5
123001    42790.5
123002    26851.5
123003    31972.5
123004    36211.5
Name: AMT_ANNUITY, Length: 123005, dtype: float64

In [25]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] >= 258025.5)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184502
Rows after: 184501


In [26]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 3191
After deleting: 3190


In [27]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 65272.5
Index of lowest outlier value in unique values: 11209
Outliers will be trimmed to: 65227.5


In [28]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

1         65227.5
15        65227.5
25        65227.5
37        65227.5
304       65227.5
           ...   
184126    65227.5
184131    65227.5
184134    65227.5
184215    65227.5
184261    65227.5
Name: AMT_ANNUITY, Length: 3190, dtype: float32

In [29]:
df[df[column_name].isna()]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
68774,157917,Cash loans,F,N,N,0,94500.0,450000.0,NaN,450000.0,Unaccompanied,Working,Lower secondary,Civil marriage,House / apartment,0.035792,-9027,-1270,-3640,-741,NaN,1,1,1,1,0,0,Laborers,2.0,2,2,MONDAY,20,0,0,0,0,0,0,Business Entity Type 1,NaN,0.727274,0.468660,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-706.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0
87443,163757,Cash loans,F,N,N,0,162000.0,296280.0,NaN,225000.0,Unaccompanied,State servant,Higher education,Married,House / apartment,0.035792,-11329,-2040,-3195,-3069,NaN,1,1,0,1,0,0,Core staff,2.0,2,2,FRIDAY,13,0,0,0,1,1,1,Government,0.336803,0.566316,0.220095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-2841.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,4.0,0
114531,379997,Cash loans,F,N,N,0,315000.0,1483231.5,NaN,1354500.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.072508,-15072,-152,-7576,-4389,NaN,1,1,1,1,1,0,Accountants,2.0,1,1,WEDNESDAY,15,0,1,1,0,1,1,Self-employed,0.593394,0.267269,NaN,0.3691,0.1891,0.9806,0.7348,0.2590,0.52,0.2241,0.5417,0.5833,0.0000,0.3009,0.3756,0.0000,0.2002,0.2216,0.1435,0.9796,0.7321,0.1590,0.3222,0.1379,0.5417,0.5833,0.0000,0.1938,0.2268,0.0000,0.0720,0.3726,0.1891,0.9806,0.7383,0.2607,0.52,0.2241,0.5417,0.5833,0.0000,0.3061,0.3824,0.0000,0.2044,reg oper spec account,block of flats,0.1860,"Stone, brick",No,0.0,0.0,0.0,0.0,-504.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
147768,187985

In [30]:
df[column_name].fillna(column_mean_without_outliers, inplace=True)
df[df[column_name].isna()]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET


In [31]:
df.shape

(184501, 122)

# AMT_GOODS_PRICE

In [32]:
column_name = "AMT_GOODS_PRICE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

AMT_GOODS_PRICE
Min value: 40500.0
Max value: 4050000.0
Std: 369130.34375
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.04003976195105268
Spearman's correlation: -0.031467356908423186
Missing value(s): 167


Lower bound: -511200.0
Upper bound: 1429200.0
Number of outlier(s): 4442
Mean with outliers: 538664.1875
Mean without outliers: 508615.4375
Less than mean: 0
More than mean: 182

Unique values: 794


In [33]:
more_than_mean

[1431000.0,
 1435500.0,
 1439248.5,
 1440000.0,
 1444500.0,
 1449000.0,
 1453500.0,
 1458000.0,
 1458859.5,
 1462500.0,
 1467000.0,
 1471500.0,
 1474663.5,
 1476000.0,
 1480500.0,
 1481535.0,
 1485000.0,
 1489500.0,
 1494000.0,
 1496893.5,
 1498500.0,
 1503000.0,
 1507500.0,
 1512000.0,
 1516500.0,
 1521000.0,
 1525500.0,
 1530000.0,
 1534500.0,
 1539000.0,
 1543500.0,
 1548000.0,
 1552500.0,
 1557000.0,
 1561500.0,
 1566000.0,
 1570500.0,
 1575000.0,
 1579500.0,
 1584000.0,
 1588500.0,
 1593000.0,
 1597500.0,
 1602000.0,
 1606500.0,
 1611000.0,
 1615500.0,
 1620000.0,
 1624500.0,
 1629000.0,
 1633500.0,
 1638000.0,
 1639431.0,
 1642500.0,
 1647000.0,
 1651500.0,
 1656000.0,
 1660500.0,
 1665000.0,
 1669500.0,
 1674000.0,
 1678500.0,
 1683000.0,
 1687500.0,
 1692000.0,
 1696500.0,
 1701000.0,
 1705500.0,
 1710000.0,
 1714500.0,
 1719000.0,
 1723500.0,
 1723666.5,
 1728000.0,
 1732500.0,
 1737000.0,
 1741500.0,
 1746000.0,
 1750500.0,
 1755000.0,
 1759500.0,
 1764000.0,
 1768500.0,
 177

In [34]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] >= 3375000.0)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184501
Rows after: 184492


In [35]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 4442
After deleting: 4433


In [36]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 1431000.0
Index of lowest outlier value in unique values: 611
Outliers will be trimmed to: 1426500.0


In [37]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

1         1426500.0
28        1426500.0
48        1426500.0
68        1426500.0
71        1426500.0
            ...    
184341    1426500.0
184385    1426500.0
184387    1426500.0
184413    1426500.0
184495    1426500.0
Name: AMT_GOODS_PRICE, Length: 4433, dtype: float32

In [38]:
df[df[column_name].isna()]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
1694,107494,Revolving loans,F,N,N,0,67500.0,202500.0,10125.0,NaN,NaN,Working,Higher education,Married,House / apartment,0.008474,-9727,-2712,-4132,-800,NaN,1,1,1,1,1,0,Accountants,2.0,2,2,WEDNESDAY,18,0,0,0,0,0,0,Trade: type 7,0.706945,0.565849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,-643.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
1950,198819,Revolving loans,F,N,Y,1,139500.0,270000.0,13500.0,NaN,NaN,Working,Secondary / secondary special,Married,House / apartment,0.010147,-13244,-109,-4947,-4958,NaN,1,1,1,1,0,0,NaN,3.0,2,2,SATURDAY,11,0,0,0,0,1,1,Business Entity Type 3,0.491681,0.217530,0.474051,0.0278,NaN,0.9876,NaN,NaN,0.00,0.1034,0.0833,NaN,0.0118,NaN,0.0312,NaN,0.0495,0.0284,NaN,0.9876,NaN,NaN,0.0000,0.1034,0.0833,NaN,0.0120,NaN,0.0325,NaN,0.0524,0.0281,NaN,0.9876,NaN,NaN,0.00,0.1034,0.0833,NaN,0.0120,NaN,0.0317,NaN,0.0505,NaN,block of flats,0.0245,"Stone, brick",No,1.0,1.0,1.0,1.0,-653.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
2236,326127,Revolving loans,F,N,Y,1,67500.0,202500.0,10125.0,NaN,NaN,Working,Secondary / secondary special,Married,House / apartment,0.010500,-13341,-6352,-2567,-4778,NaN,1,1,1,1,1,0,Accountants,3.0,3,3,SUNDAY,13,0,0,0,0,0,0,Other,NaN,0.552557,0.694093,0.0464,0.0537,0.9896,0.8776,0.0409,0.00,0.1034,0.1667,0.2083,0.0538,0.0227,0.0360,0.0116,0.0049,0.0315,0.0346,0.9881,0.8824,0.0413,0.0000,0.0690,0.1667,0.2083,0.0405,0.0248,0.0175,0.0117,0.0000,0.0468,0.0537,0.9896,0.8792,0.0411,0.00,0.1034,0.1667,0.2083,0.0547,0.0231,0.0366,0.0116,0.0050,reg oper account,block of flats,0.0224,"Stone, brick",No,0.0,0.0,0.0,0.0,-305.0,0,0,0,0,0,0

In [39]:
df[column_name].fillna(column_mean_without_outliers, inplace=True)
df[df[column_name].isna()]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET


In [40]:
df.shape

(184492, 122)

# REGION_POPULATION_RELATIVE

In [41]:
column_name = "REGION_POPULATION_RELATIVE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

REGION_POPULATION_RELATIVE
Min value: 0.0002899999963119626
Max value: 0.07250799983739853
Std: 0.01381777785718441
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.03900276102491812
Spearman's correlation: -0.03116728795977117
Missing value(s): 0


Lower bound: -0.02171089937910437
Upper bound: 0.060379899758845564
Number of outlier(s): 5009
Mean with outliers: 0.020868394523859024
Mean without outliers: 0.019427238032221794
Less than mean: 0
More than mean: 1

Unique values: 81


In [42]:
more_than_mean

[0.07250799983739853]

In [43]:
lowest_outlier_value = more_than_mean[0]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value - 1]}")

Lowest outlier value in unique values: 0.07250799983739853
Index of lowest outlier value in unique values: 80
Outliers will be trimmed to: 0.046220000833272934


In [44]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value - 1]
df[column_name][index_of_outliers]

70        0.04622
97        0.04622
100       0.04622
148       0.04622
173       0.04622
           ...   
184134    0.04622
184251    0.04622
184386    0.04622
184475    0.04622
184491    0.04622
Name: REGION_POPULATION_RELATIVE, Length: 5009, dtype: float32

In [45]:
df.shape

(184492, 122)

# DAYS_BIRTH	

In [46]:
column_name = "DAYS_BIRTH"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

DAYS_BIRTH
Min value: -25201
Max value: -7489
Std: 4361.563038278779
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.08083513952151207
Spearman's correlation: 0.0809699012824057
Missing value(s): 0


Lower bound: -32020.699999999997
Upper bound: -72.30000000000109
Number of outlier(s): 0
Mean with outliers: -16036.866563319818
Mean without outliers: -16036.866563319818
Less than mean: 0
More than mean: 0

Unique values: 17334


In [47]:
df.shape

(184492, 122)

# DAYS_EMPLOYED

In [48]:
column_name = "DAYS_EMPLOYED"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

DAYS_EMPLOYED
Min value: -17912
Max value: 365243
Std: 141299.49204359748
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.04687061059284469
Spearman's correlation: 0.023889179823989515
Missing value(s): 0


Lower bound: -6952.599999999999
Upper bound: 3906.5999999999995
Number of outlier(s): 41863
Mean with outliers: 63843.58187888906
Mean without outliers: -1962.5269755800014
Less than mean: 4508
More than mean: 1

Unique values: 11403


In [49]:
less_than_mean[:50]

[-17912,
 -17583,
 -17531,
 -17522,
 -17170,
 -17139,
 -16849,
 -16836,
 -16767,
 -16678,
 -16632,
 -16607,
 -16554,
 -16538,
 -16495,
 -16492,
 -16452,
 -16429,
 -16424,
 -16375,
 -16365,
 -16364,
 -16360,
 -16358,
 -16348,
 -16343,
 -16314,
 -16310,
 -16308,
 -16304,
 -16266,
 -16265,
 -16263,
 -16221,
 -16160,
 -16142,
 -16135,
 -16133,
 -16121,
 -16113,
 -16069,
 -16061,
 -16032,
 -15943,
 -15871,
 -15845,
 -15837,
 -15834,
 -15791,
 -15727]

In [50]:
df[df[column_name] <= -17139]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
70999,140766,Cash loans,F,Y,Y,0,135000.0,284400.0,16011.0,225000.0,Unaccompanied,Working,Secondary / secondary special,Widow,House / apartment,0.020246,-24254,-17531,-1850,-4263,64.0,1,1,0,1,1,0,Managers,1.0,3,3,FRIDAY,7,0,0,0,0,0,0,Medicine,NaN,0.420840,0.502878,0.0495,0.0708,0.9975,NaN,NaN,0.0,0.1379,0.0833,NaN,0.0362,NaN,0.0352,NaN,0.0000,0.0504,0.0735,0.9975,NaN,NaN,0.0,0.1379,0.0833,NaN,0.0370,NaN,0.0366,NaN,0.0000,0.0500,0.0708,0.9975,NaN,NaN,0.0,0.1379,0.0833,NaN,0.0368,NaN,0.0358,NaN,0.0000,NaN,block of flats,0.0306,Others,No,1.0,0.0,1.0,0.0,-681.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,1.0,0
89379,371247,Cash loans,F,N,N,0,171000.0,284400.0,16456.5,225000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.020713,-24794,-17522,-2503,-3990,NaN,1,1,0,1,0,0,High skill tech staff,2.0,3,3,FRIDAY,15,0,0,0,0,1,1,Trade: type 6,NaN,0.258225,0.332851,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,-105.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0
96947,266124,Cash loans,F,N,Y,0,135000.0,148707.0,14485.5,139500.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.031329,-25101,-17139,-13828,-4455,NaN,1,1,0,1,0,0,Core staff,2.0,2,2,MONDAY,15,0,0,0,0,0,0,Medicine,NaN,0.644672,0.652897,0.0825,0.0899,0.9811,0.7416,NaN,0.0,0.2069,0.1667,0.0417,0.0707,0.0672,0.0784,0.0000,0.0000,0.0840,0.0933,0.9811,0.7517,NaN,0.0,0.2069,0.1667,0.0417,0.0723,0.0735,0.0816,0.000,0.0000,0.0833,0.0899,0.9811,0.7451,NaN,0.0,0.2069,0.1667,0.0417,0.0719,0.0684,0.0798,0.0000,0.0000,org spec account,block of flats,0.0616,Panel,No,3.0,0.0,3.0,0.0,

In [51]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] <= -17139)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184492
Rows after: 184486


In [52]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 41863
After deleting: 41857


In [53]:
more_than_mean

[365243]

In [54]:
unique_values[-10:]

array([    -9,     -8,     -6,     -5,     -4,     -3,     -2,     -1,
            0, 365243], dtype=int32)

In [55]:
indices_of_more_than_mean = df[df[column_name] == more_than_mean[0]].index
print(f"Number of more than mean outliers: {len(indices_of_more_than_mean)}")

Number of more than mean outliers: 33236


In [56]:
df[column_name].replace([365243], 0, inplace=True)

In [57]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(indices_of_more_than_mean)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 41857
After deleting: 8621


In [58]:
less_than_mean

[-17912,
 -17583,
 -17531,
 -17522,
 -17170,
 -17139,
 -16849,
 -16836,
 -16767,
 -16678,
 -16632,
 -16607,
 -16554,
 -16538,
 -16495,
 -16492,
 -16452,
 -16429,
 -16424,
 -16375,
 -16365,
 -16364,
 -16360,
 -16358,
 -16348,
 -16343,
 -16314,
 -16310,
 -16308,
 -16304,
 -16266,
 -16265,
 -16263,
 -16221,
 -16160,
 -16142,
 -16135,
 -16133,
 -16121,
 -16113,
 -16069,
 -16061,
 -16032,
 -15943,
 -15871,
 -15845,
 -15837,
 -15834,
 -15791,
 -15727,
 -15713,
 -15691,
 -15687,
 -15676,
 -15661,
 -15632,
 -15629,
 -15578,
 -15569,
 -15568,
 -15543,
 -15542,
 -15530,
 -15524,
 -15499,
 -15474,
 -15473,
 -15439,
 -15427,
 -15412,
 -15396,
 -15360,
 -15348,
 -15342,
 -15338,
 -15334,
 -15303,
 -15290,
 -15285,
 -15277,
 -15238,
 -15227,
 -15210,
 -15202,
 -15183,
 -15181,
 -15154,
 -15153,
 -15147,
 -15137,
 -15116,
 -15107,
 -15095,
 -15094,
 -15084,
 -15080,
 -15072,
 -15066,
 -15059,
 -15057,
 -15052,
 -15051,
 -15034,
 -15019,
 -14991,
 -14988,
 -14981,
 -14977,
 -14966,
 -14955,
 -14928,
 

In [59]:
lowest_outlier_value = less_than_mean[-1]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value + 1]}")

Lowest outlier value in unique values: -6953
Index of lowest outlier value in unique values: 4507
Outliers will be trimmed to: -6952


In [60]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value + 1]
df[column_name][index_of_outliers]

26       -6952
45       -6952
57       -6952
61       -6952
62       -6952
          ... 
184323   -6952
184367   -6952
184369   -6952
184392   -6952
184411   -6952
Name: DAYS_EMPLOYED, Length: 8621, dtype: int32

In [61]:
df.shape

(184486, 122)

# DAYS_REGISTRATION

In [62]:
column_name = "DAYS_REGISTRATION"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

DAYS_REGISTRATION
Min value: -23416
Max value: 0
Std: 3524.967102153562
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.04401606935219487
Spearman's correlation: 0.04210780232766342
Missing value(s): 0


Lower bound: -16812.199999999997
Upper bound: 7326.199999999999
Number of outlier(s): 144
Mean with outliers: -4984.907846665872
Mean without outliers: -4974.603709409685
Less than mean: 138
More than mean: 0

Unique values: 15002


In [63]:
less_than_mean

[-23416,
 -22928,
 -22858,
 -22392,
 -21887,
 -21865,
 -21244,
 -21146,
 -20841,
 -20840,
 -20838,
 -20741,
 -20701,
 -20623,
 -20569,
 -20514,
 -20452,
 -20447,
 -20096,
 -19706,
 -19695,
 -19660,
 -19579,
 -19513,
 -19504,
 -19463,
 -19370,
 -19265,
 -19244,
 -19183,
 -19170,
 -19134,
 -19040,
 -18926,
 -18864,
 -18758,
 -18718,
 -18645,
 -18579,
 -18417,
 -18376,
 -18350,
 -18228,
 -18208,
 -18132,
 -18130,
 -18125,
 -18123,
 -18053,
 -18042,
 -18034,
 -17995,
 -17965,
 -17959,
 -17951,
 -17884,
 -17879,
 -17866,
 -17783,
 -17781,
 -17777,
 -17747,
 -17736,
 -17714,
 -17709,
 -17692,
 -17663,
 -17620,
 -17619,
 -17596,
 -17588,
 -17565,
 -17552,
 -17536,
 -17524,
 -17522,
 -17513,
 -17501,
 -17481,
 -17480,
 -17442,
 -17441,
 -17430,
 -17416,
 -17399,
 -17394,
 -17389,
 -17379,
 -17338,
 -17313,
 -17302,
 -17298,
 -17278,
 -17274,
 -17253,
 -17252,
 -17248,
 -17247,
 -17236,
 -17215,
 -17205,
 -17201,
 -17190,
 -17188,
 -17176,
 -17170,
 -17141,
 -17130,
 -17122,
 -17121,
 -17103,
 

In [64]:
df[df[column_name] <= -21146]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
25186,210692,Cash loans,F,N,Y,0,112500.0,733500.0,21577.5,733500.0,Unaccompanied,Pensioner,Lower secondary,Married,House / apartment,0.014520,-21898,0,-21887,-4159,NaN,1,0,0,1,0,0,NaN,2.0,2,2,FRIDAY,8,0,0,0,0,0,0,XNA,NaN,0.231888,0.586740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,-1845.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0,0
59972,427020,Cash loans,M,N,Y,0,112500.0,1042560.0,34587.0,900000.0,Family,Pensioner,Secondary / secondary special,Married,House / apartment,0.025164,-21146,0,-21146,-966,NaN,1,0,0,1,0,0,NaN,2.0,2,2,THURSDAY,11,0,0,0,0,0,0,XNA,NaN,0.162192,0.315472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,-220.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
133763,344459,Cash loans,F,N,Y,0,180000.0,1350000.0,37255.5,1350000.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.015221,-22858,0,-22858,-5106,NaN,1,0,0,1,0,0,NaN,2.0,2,2,FRIDAY,10,0,0,0,0,0,0,XNA,NaN,0.481876,0.681706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,0.0,5.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,2.0,2.0,0
140491,125615,Cash loans,F,N,Y,0,39600.0,283585.5,17478.0,256500.0,Unaccompanied,Pensioner,Secondary / secondary special,Married,House / apartment,0.020713,-22220,0,-21244,-4131,NaN,1,0,0,1,0,0,NaN,2.0,3,3,SUNDAY,14,0,0,0,0,

In [65]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] <= -21146)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184486
Rows after: 184478


In [66]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 144
After deleting: 136


In [67]:
lowest_outlier_value = less_than_mean[-1]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value + 1]}")

Lowest outlier value in unique values: -16820
Index of lowest outlier value in unique values: 137
Outliers will be trimmed to: -16787


In [68]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value + 1]
df[column_name][index_of_outliers]

2075     -16787
4266     -16787
7666     -16787
13221    -16787
13810    -16787
          ...  
177023   -16787
177051   -16787
177647   -16787
179995   -16787
180274   -16787
Name: DAYS_REGISTRATION, Length: 136, dtype: int16

In [69]:
df.shape

(184478, 122)

# DAYS_ID_PUBLISH

In [70]:
column_name = "DAYS_ID_PUBLISH"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

DAYS_ID_PUBLISH
Min value: -6383
Max value: 0
Std: 1509.9530593269649
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.05105769340013298
Spearman's correlation: 0.052132926850645515
Missing value(s): 0


Lower bound: -8681.599999999999
Upper bound: 2661.5999999999995
Number of outlier(s): 0
Mean with outliers: -2994.0334674053274
Mean without outliers: -2994.0334674053274
Less than mean: 0
More than mean: 0

Unique values: 6086


In [71]:
df.shape

(184478, 122)

# DAYS_LAST_PHONE_CHANGE

In [72]:
column_name = "DAYS_LAST_PHONE_CHANGE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

DAYS_LAST_PHONE_CHANGE
Min value: -4185.0
Max value: 0.0
Std: 827.4054166062182
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.05271481218730734
Spearman's correlation: 0.05091045623929364
Missing value(s): 1


Lower bound: -3773.9
Upper bound: 1932.9
Number of outlier(s): 56
Mean with outliers: -962.0770665177773
Mean without outliers: -961.1749312713845
Less than mean: 55
More than mean: 0

Unique values: 3683


In [73]:
df[column_name].fillna(column_mean_without_outliers, inplace=True)

In [74]:
less_than_mean

[-4185.0,
 -4173.0,
 -4153.0,
 -4131.0,
 -4128.0,
 -4121.0,
 -4115.0,
 -4079.0,
 -4070.0,
 -4066.0,
 -4051.0,
 -4043.0,
 -4033.0,
 -4026.0,
 -4021.0,
 -4020.0,
 -4016.0,
 -4002.0,
 -3999.0,
 -3988.0,
 -3983.0,
 -3978.0,
 -3967.0,
 -3960.0,
 -3949.0,
 -3938.0,
 -3926.0,
 -3917.0,
 -3905.0,
 -3900.0,
 -3899.0,
 -3882.0,
 -3862.0,
 -3859.0,
 -3857.0,
 -3856.0,
 -3851.0,
 -3845.0,
 -3841.0,
 -3840.0,
 -3824.0,
 -3818.0,
 -3811.0,
 -3809.0,
 -3808.0,
 -3807.0,
 -3802.0,
 -3797.0,
 -3793.0,
 -3791.0,
 -3788.0,
 -3784.0,
 -3783.0,
 -3782.0,
 -3776.0]

In [75]:
df[df[column_name] <= -4115.0]

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,TARGET
10923,259118,Cash loans,F,Y,Y,1,270000.0,1024740.0,52452.0,900000.0,Family,Working,Secondary / secondary special,Married,House / apartment,0.046220,-12412,-224,-6520,-4123,3.0,1,1,0,1,1,1,Accountants,3.0,1,1,MONDAY,15,0,0,0,0,0,0,Business Entity Type 3,0.830061,0.736995,0.253963,0.6304,0.4389,0.9806,NaN,NaN,0.74,0.4310,0.4792,NaN,0.4872,NaN,0.7114,NaN,0.0365,0.4853,0.3523,0.9806,NaN,NaN,0.5236,0.4138,0.3333,NaN,0.3661,NaN,0.4758,NaN,0.0027,0.6365,0.4389,0.9806,NaN,NaN,0.74,0.4310,0.4792,NaN,0.4957,NaN,0.7242,NaN,0.0373,NaN,block of flats,0.3597,Panel,No,1.0,0.0,1.0,0.0,-4185.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0
54292,440919,Cash loans,F,N,Y,0,270000.0,288562.5,22927.5,261000.0,Unaccompanied,Commercial associate,Higher education,Single / not married,House / apartment,0.046220,-21177,-2870,-14935,-2367,NaN,1,1,0,1,1,1,Accountants,1.0,1,1,TUESDAY,12,0,0,0,0,0,0,Business Entity Type 3,NaN,0.756422,0.086162,0.2467,0.1259,0.9786,0.7076,0.0694,0.40,0.1724,0.4583,0.5000,0.0000,0.2009,0.2445,0.0013,0.0001,0.2511,0.1281,0.9786,0.7190,0.0691,0.4028,0.1724,0.4583,0.5000,0.0000,0.2195,0.2543,0.0,0.0000,0.2488,0.1270,0.9786,0.7115,0.0697,0.40,0.1724,0.4583,0.5000,0.0000,0.2044,0.2488,0.0000,0.0000,reg oper account,block of flats,0.1920,Panel,No,0.0,0.0,0.0,0.0,-4173.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0,0
97961,291970,Cash loans,F,N,N,0,171000.0,640080.0,31261.5,450000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Separated,House / apartment,0.046220,-17839,-4449,-11883,-1131,NaN,1,1,0,1,0,0,NaN,1.0,1,1,SATURDAY,10,0,0,0,0,0,0,Business Entity Type 1,NaN,0.722643,0.782608,0.0866,0.0384,0.9771,0.6872,0.0000,0.08,0.0345,0.4583,0.5000,0.0000,0.0706,0.0727,0.0000,0.0000,0.0882,0.0398,0.9772,0.6994,0.0000,0.0

In [76]:
print(f"Rows before: {df.shape[0]}")
to_delete_indices = df[(df[column_name] <= -4115.0)].index
df.drop(index=to_delete_indices, inplace=True)
print(f"Rows after: {df.shape[0]}")

Rows before: 184478
Rows after: 184471


In [77]:
print(f"Before deleting: {len(index_of_outliers)}")
index_of_outliers = index_of_outliers.drop(to_delete_indices)
print(f"After deleting: {len(index_of_outliers)}")

Before deleting: 56
After deleting: 49


In [78]:
lowest_outlier_value = less_than_mean[-1]
index_lowest_outlier_value = np.where(unique_values == lowest_outlier_value)[0][0]
print(f"Lowest outlier value in unique values: {lowest_outlier_value}")
print(f"Index of lowest outlier value in unique values: {index_lowest_outlier_value}")
print(f"Outliers will be trimmed to: {unique_values[index_lowest_outlier_value + 1]}")

Lowest outlier value in unique values: -3776.0
Index of lowest outlier value in unique values: 54
Outliers will be trimmed to: -3772.0


In [79]:
df.loc[index_of_outliers, column_name] = unique_values[index_lowest_outlier_value + 1]
df[column_name][index_of_outliers]

2613     -3772.0
5865     -3772.0
15592    -3772.0
16357    -3772.0
18372    -3772.0
18674    -3772.0
18773    -3772.0
23241    -3772.0
25272    -3772.0
30063    -3772.0
41853    -3772.0
43350    -3772.0
46450    -3772.0
47723    -3772.0
48191    -3772.0
57683    -3772.0
63187    -3772.0
64497    -3772.0
65204    -3772.0
71134    -3772.0
73859    -3772.0
80015    -3772.0
83528    -3772.0
89063    -3772.0
89720    -3772.0
90146    -3772.0
98343    -3772.0
106670   -3772.0
106735   -3772.0
115531   -3772.0
118041   -3772.0
118244   -3772.0
123604   -3772.0
125505   -3772.0
134148   -3772.0
136105   -3772.0
136994   -3772.0
143425   -3772.0
149767   -3772.0
151456   -3772.0
151925   -3772.0
153494   -3772.0
156135   -3772.0
157617   -3772.0
160093   -3772.0
170220   -3772.0
171258   -3772.0
171309   -3772.0
179841   -3772.0
Name: DAYS_LAST_PHONE_CHANGE, dtype: float64

In [80]:
df.shape

(184471, 122)

# OWN_CAR_AGE

In [81]:
column_name = "OWN_CAR_AGE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

OWN_CAR_AGE
Min value: 0.0
Max value: 91.0
Std: 11.929660022533682
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.03835104062100958
Spearman's correlation: 0.05156432022760307
Missing value(s): 121614


Lower bound: -12.0
Upper bound: 32.0
Number of outlier(s): 2686
Mean with outliers: 12.049970568114928
Mean without outliers: 10.02517824201027
Less than mean: 0
More than mean: 27

Unique values: 61


In [82]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 121)

In [83]:
df.shape

(184471, 121)

# FLAG_MOBIL

In [84]:
column_name = "FLAG_MOBIL"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_MOBIL
Min value: 0
Max value: 1
Std: 0.0023282839747568902
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.0006899658645448236
Spearman's correlation: 0.0006899658645448184
Missing value(s): 0


Lower bound: 1.0
Upper bound: 1.0
Number of outlier(s): 1
Mean with outliers: 0.9999945790937329
Mean without outliers: 1.0
Less than mean: 1
More than mean: 0

Unique values: 2


In [85]:
df[column_name].value_counts()

1    184470
0         1
Name: FLAG_MOBIL, dtype: int64

In [86]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 120)

# FLAG_EMP_PHONE

In [87]:
column_name = "FLAG_EMP_PHONE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_EMP_PHONE
Min value: 0
Max value: 1
Std: 0.3843158637922268
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.047938604569785576
Spearman's correlation: 0.04793860456978545
Missing value(s): 0


Lower bound: 1.0
Upper bound: 1.0
Number of outlier(s): 33233
Mean with outliers: 0.8198470220251421
Mean without outliers: 1.0
Less than mean: 1
More than mean: 0

Unique values: 2


In [88]:
df[column_name].value_counts()

1    151238
0     33233
Name: FLAG_EMP_PHONE, dtype: int64

In [89]:
df.shape

(184471, 120)

# FLAG_WORK_PHONE

In [90]:
column_name = "FLAG_WORK_PHONE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_WORK_PHONE
Min value: 0
Max value: 1
Std: 0.39931560844313346
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.02965989140854969
Spearman's correlation: 0.029659891408549844
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 36726
Mean with outliers: 0.19908820356587215
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [91]:
df[column_name].value_counts()

0    147745
1     36726
Name: FLAG_WORK_PHONE, dtype: int64

In [92]:
df.shape

(184471, 120)

# FLAG_CONT_MOBILE

In [93]:
column_name = "FLAG_CONT_MOBILE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_CONT_MOBILE
Min value: 0
Max value: 1
Std: 0.04419420898663632
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.0005145945831585461
Spearman's correlation: 0.0005145945831584912
Missing value(s): 0


Lower bound: 1.0
Upper bound: 1.0
Number of outlier(s): 361
Mean with outliers: 0.9980430528375733
Mean without outliers: 1.0
Less than mean: 1
More than mean: 0

Unique values: 2


In [94]:
df[column_name].value_counts()

1    184110
0       361
Name: FLAG_CONT_MOBILE, dtype: int64

In [95]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 119)

# FLAG_PHONE

In [96]:
column_name = "FLAG_PHONE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_PHONE
Min value: 0
Max value: 1
Std: 0.4497857186690046
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.023304198269731066
Spearman's correlation: -0.02330419826973088
Missing value(s): 0


Lower bound: -1.7
Upper bound: 2.7
Number of outlier(s): 0
Mean with outliers: 0.28161065967008364
Mean without outliers: 0.28161065967008364
Less than mean: 0
More than mean: 0

Unique values: 2


In [97]:
df[column_name].value_counts()

0    132522
1     51949
Name: FLAG_PHONE, dtype: int64

In [98]:
df.shape

(184471, 119)

# FLAG_EMAIL

In [99]:
column_name = "FLAG_EMAIL"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_EMAIL
Min value: 0
Max value: 1
Std: 0.23112871622294937
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.0016604636055905888
Spearman's correlation: -0.0016604636055905545
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 10446
Mean with outliers: 0.056626786866228296
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [100]:
df[column_name].value_counts()

0    174025
1     10446
Name: FLAG_EMAIL, dtype: int64

In [101]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 118)

# FLAG_DOCUMENT_2	

In [102]:
column_name = "FLAG_DOCUMENT_2"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_2
Min value: 0
Max value: 1
Std: 0.006159960198056295
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.0046353579874245775
Spearman's correlation: 0.004635357987424542
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 7
Mean with outliers: 3.794634386976815e-05
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [103]:
df[column_name].value_counts()

0    184464
1         7
Name: FLAG_DOCUMENT_2, dtype: int64

In [104]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 117)

# FLAG_DOCUMENT_3

In [105]:
column_name = "FLAG_DOCUMENT_3"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_3
Min value: 0
Max value: 1
Std: 0.45448357689068325
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.04772348063411606
Spearman's correlation: 0.04772348063411599
Missing value(s): 0


Lower bound: -1.7
Upper bound: 2.7
Number of outlier(s): 0
Mean with outliers: 0.7084365564235029
Mean without outliers: 0.7084365564235029
Less than mean: 0
More than mean: 0

Unique values: 2


In [106]:
df[column_name].value_counts()

1    130686
0     53785
Name: FLAG_DOCUMENT_3, dtype: int64

In [107]:
df.shape

(184471, 117)

# FLAG_DOCUMENT_4

In [108]:
column_name = "FLAG_DOCUMENT_4"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_4
Min value: 0
Max value: 1
Std: 0.009599344427845154
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.002844925517529631
Spearman's correlation: -0.0028449255175296466
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 17
Mean with outliers: 9.215540654086551e-05
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [109]:
df[column_name].value_counts()

0    184454
1        17
Name: FLAG_DOCUMENT_4, dtype: int64

In [110]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 116)

# FLAG_DOCUMENT_5

In [111]:
column_name = "FLAG_DOCUMENT_5"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_5
Min value: 0
Max value: 1
Std: 0.1238852093981648
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.00018859556407871687
Spearman's correlation: -0.00018859556407882112
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 2876
Mean with outliers: 0.015590526424207598
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [112]:
df[column_name].value_counts()

0    181595
1      2876
Name: FLAG_DOCUMENT_5, dtype: int64

In [113]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 115)

# FLAG_DOCUMENT_6

In [114]:
column_name = "FLAG_DOCUMENT_6"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_6
Min value: 0
Max value: 1
Std: 0.2837645868585677
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.031788462725684044
Spearman's correlation: -0.03178846272568396
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 16293
Mean with outliers: 0.08832282581001892
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [115]:
df[column_name].value_counts()

0    168178
1     16293
Name: FLAG_DOCUMENT_6, dtype: int64

In [116]:
df.shape

(184471, 115)

# FLAG_DOCUMENT_7

In [117]:
column_name = "FLAG_DOCUMENT_7"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_7
Min value: 0
Max value: 1
Std: 0.01337381302370631
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.0004999030996406152
Spearman's correlation: 0.0004999030996405657
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 33
Mean with outliers: 0.00017888990681462128
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [118]:
df[column_name].value_counts()

0    184438
1        33
Name: FLAG_DOCUMENT_7, dtype: int64

In [119]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 114)

# FLAG_DOCUMENT_8

In [120]:
column_name = "FLAG_DOCUMENT_8"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_8
Min value: 0
Max value: 1
Std: 0.27485523416265367
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.008828192922985824
Spearman's correlation: -0.008828192922986
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 15186
Mean with outliers: 0.08232188257232845
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [121]:
df[column_name].value_counts()

0    169285
1     15186
Name: FLAG_DOCUMENT_8, dtype: int64

In [122]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 113)

# FLAG_DOCUMENT_9

In [123]:
column_name = "FLAG_DOCUMENT_9"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_9
Min value: 0
Max value: 1
Std: 0.061789260817194554
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.002600500764575625
Spearman's correlation: -0.002600500764575693
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 707
Mean with outliers: 0.003832580730846583
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [124]:
df[column_name].value_counts()

0    183764
1       707
Name: FLAG_DOCUMENT_9, dtype: int64

In [125]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 112)

# FLAG_DOCUMENT_10

In [126]:
column_name = "FLAG_DOCUMENT_10"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_10
Min value: 0
Max value: 1
Std: 0.0046565300849269985
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.0013799429500081766
Spearman's correlation: -0.0013799429500081643
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 4
Mean with outliers: 2.1683625068438943e-05
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [127]:
df[column_name].value_counts()

0    184467
1         4
Name: FLAG_DOCUMENT_10, dtype: int64

In [128]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 111)

# FLAG_DOCUMENT_11

In [129]:
column_name = "FLAG_DOCUMENT_11"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_11
Min value: 0
Max value: 1
Std: 0.06299613040489399
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.0032646964140684745
Spearman's correlation: -0.0032646964140685608
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 735
Mean with outliers: 0.003984366106325655
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [130]:
df[column_name].value_counts()

0    183736
1       735
Name: FLAG_DOCUMENT_11, dtype: int64

In [131]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 110)

# FLAG_DOCUMENT_12

In [132]:
column_name = "FLAG_DOCUMENT_12"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_12
Min value: 0
Max value: 0
Std: 0.0
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: nan
Spearman's correlation: nan
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 0
Mean with outliers: 0.0
Mean without outliers: 0.0
Less than mean: 0
More than mean: 0

Unique values: 1


/home/dhruv/miniconda3/envs/test/lib/python3.9/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [133]:
df[column_name].value_counts()

0    184471
Name: FLAG_DOCUMENT_12, dtype: int64

In [134]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 109)

# FLAG_DOCUMENT_13

In [135]:
column_name = "FLAG_DOCUMENT_13"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_13
Min value: 0
Max value: 1
Std: 0.061000609272621235
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.012272773173975128
Spearman's correlation: -0.012272773173975146
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 689
Mean with outliers: 0.0037350044180386077
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [136]:
df[column_name].value_counts()

0    183782
1       689
Name: FLAG_DOCUMENT_13, dtype: int64

In [137]:
df.shape

(184471, 109)

# FLAG_DOCUMENT_14

In [138]:
column_name = "FLAG_DOCUMENT_14"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_14
Min value: 0
Max value: 1
Std: 0.05397542918356786
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.010143130967018101
Spearman's correlation: -0.01014313096701808
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 539
Mean with outliers: 0.0029218684779721473
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [139]:
df[column_name].value_counts()

0    183932
1       539
Name: FLAG_DOCUMENT_14, dtype: int64

In [140]:
df.shape

(184471, 109)

# FLAG_DOCUMENT_15

In [141]:
column_name = "FLAG_DOCUMENT_15"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_15
Min value: 0
Max value: 1
Std: 0.03597116958817521
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.007907488930994064
Spearman's correlation: -0.007907488930994067
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 239
Mean with outliers: 0.0012955965978392267
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [142]:
df[column_name].value_counts()

0    184232
1       239
Name: FLAG_DOCUMENT_15, dtype: int64

In [143]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 108)

# FLAG_DOCUMENT_16

In [144]:
column_name = "FLAG_DOCUMENT_16"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_16
Min value: 0
Max value: 1
Std: 0.09789146290633532
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.010794090753378435
Spearman's correlation: -0.01079409075337838
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 1785
Mean with outliers: 0.009676317686790878
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [145]:
df[column_name].value_counts()

0    182686
1      1785
Name: FLAG_DOCUMENT_16, dtype: int64

In [146]:
df.shape

(184471, 108)

# FLAG_DOCUMENT_17

In [147]:
column_name = "FLAG_DOCUMENT_17"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_17
Min value: 0
Max value: 1
Std: 0.016461267166125015
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.003670585300943959
Spearman's correlation: -0.0036705853009439
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 50
Mean with outliers: 0.00027104531335548677
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [148]:
df[column_name].value_counts()

0    184421
1        50
Name: FLAG_DOCUMENT_17, dtype: int64

In [149]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 107)

# FLAG_DOCUMENT_18

In [150]:
column_name = "FLAG_DOCUMENT_18"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_18
Min value: 0
Max value: 1
Std: 0.08830952284484865
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.008575351219530937
Spearman's correlation: -0.008575351219530954
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 1450
Mean with outliers: 0.007860314087309117
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [151]:
df[column_name].value_counts()

0    183021
1      1450
Name: FLAG_DOCUMENT_18, dtype: int64

In [152]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 106)

# FLAG_DOCUMENT_19

In [153]:
column_name = "FLAG_DOCUMENT_19"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_19
Min value: 0
Max value: 1
Std: 0.025068568220411012
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: 0.0005044863236993757
Spearman's correlation: 0.0005044863236992872
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 116
Mean with outliers: 0.0006288251269847293
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [154]:
df[column_name].value_counts()

0    184355
1       116
Name: FLAG_DOCUMENT_19, dtype: int64

In [155]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 105)

# FLAG_DOCUMENT_20

In [156]:
column_name = "FLAG_DOCUMENT_20"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_20
Min value: 0
Max value: 1
Std: 0.02350807735049022
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.00019830697151498826
Spearman's correlation: -0.000198306971514941
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 102
Mean with outliers: 0.000552932439245193
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [157]:
df[column_name].value_counts()

0    184369
1       102
Name: FLAG_DOCUMENT_20, dtype: int64

In [158]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 104)

# FLAG_DOCUMENT_21

In [159]:
column_name = "FLAG_DOCUMENT_21"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

FLAG_DOCUMENT_21
Min value: 0
Max value: 1
Std: 0.01803192580472258
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.0009310617282098989
Spearman's correlation: -0.0009310617282099272
Missing value(s): 0


Lower bound: 0.0
Upper bound: 0.0
Number of outlier(s): 60
Mean with outliers: 0.00032525437602658414
Mean without outliers: 0.0
Less than mean: 0
More than mean: 1

Unique values: 2


In [160]:
df[column_name].value_counts()

0    184411
1        60
Name: FLAG_DOCUMENT_21, dtype: int64

In [161]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 103)

# EXT_SOURCE_1

In [162]:
column_name = "EXT_SOURCE_1"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

EXT_SOURCE_1
Min value: 0.014568132348358631
Max value: 0.9626927971839905
Std: 0.21067199110984802
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.15613809310874063
Spearman's correlation: -0.15183996745389422
Missing value(s): 104055


Lower bound: -0.24270922094583514
Upper bound: 1.2529294654726981
Number of outlier(s): 0
Mean with outliers: 0.5025351643562317
Mean without outliers: 0.5025351643562317
Less than mean: 0
More than mean: 0

Unique values: 73045


In [163]:
df.drop(columns=[column_name], inplace=True)
df.shape

(184471, 102)

# EXT_SOURCE_2

In [ ]:
column_name = "EXT_SOURCE_2"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

EXT_SOURCE_2
Min value: 1.315955614700215e-06
Max value: 0.8549996614456177
Std: 0.1908775269985199
Min std: 0.0
Max std:  402198.1853209468

Pearson's correlation: -0.16130680106199877


In [ ]:
df[column_name].fillna(df[column_name].median(), inplace=True)
df[column_name].isna().sum()

In [ ]:
df.shape

# EXT_SOURCE_3

In [ ]:
column_name = "EXT_SOURCE_3"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# CNT_FAM_MEMBERS

In [ ]:
column_name = "CNT_FAM_MEMBERS"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# REGION_RATING_CLIENT

In [ ]:
column_name = "REGION_RATING_CLIENT"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
unique_values

In [ ]:
df.shape

# REGION_RATING_CLIENT_W_CITY

In [ ]:
column_name = "REGION_RATING_CLIENT_W_CITY"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
unique_values

In [ ]:
df.shape

# HOUR_APPR_PROCESS_START

In [ ]:
column_name = "HOUR_APPR_PROCESS_START"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
unique_values

In [ ]:
df[column_name].value_counts()

In [ ]:
df[column_name].replace([21, 22, 23, 0, 1, 2], 21, inplace=True)
df[column_name].value_counts()

In [ ]:
df.shape

# REG_REGION_NOT_LIVE_REGION

In [ ]:
column_name = "REG_REGION_NOT_LIVE_REGION"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# REG_REGION_NOT_WORK_REGION

In [ ]:
column_name = "REG_REGION_NOT_WORK_REGION"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LIVE_REGION_NOT_WORK_REGION

In [ ]:
column_name = "LIVE_REGION_NOT_WORK_REGION"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# REG_CITY_NOT_LIVE_CITY

In [ ]:
column_name = "REG_CITY_NOT_LIVE_CITY"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.shape

# REG_CITY_NOT_WORK_CITY

In [ ]:
column_name = "REG_CITY_NOT_WORK_CITY"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.shape

# LIVE_CITY_NOT_WORK_CITY

In [ ]:
column_name = "LIVE_CITY_NOT_WORK_CITY"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.shape

# APARTMENTS_AVG

In [ ]:
column_name = "APARTMENTS_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# BASEMENTAREA_AVG

In [ ]:
column_name = "BASEMENTAREA_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# YEARS_BEGINEXPLUATATION_AVG

In [ ]:
column_name = "YEARS_BEGINEXPLUATATION_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# YEARS_BUILD_AVG 

In [ ]:
column_name = "YEARS_BUILD_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# COMMONAREA_AVG

In [ ]:
column_name = "COMMONAREA_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# ELEVATORS_AVG

In [ ]:
column_name = "ELEVATORS_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# ENTRANCES_AVG

In [ ]:
column_name = "ENTRANCES_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# FLOORSMAX_AVG

In [ ]:
column_name = "FLOORSMAX_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# FLOORSMIN_AVG

In [ ]:
column_name = "FLOORSMIN_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LANDAREA_AVG

In [ ]:
column_name = "LANDAREA_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LIVINGAPARTMENTS_AVG

In [ ]:
column_name = "LIVINGAPARTMENTS_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LIVINGAREA_AVG

In [ ]:
column_name = "LIVINGAREA_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NONLIVINGAPARTMENTS_AVG

In [ ]:
column_name = "NONLIVINGAPARTMENTS_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NONLIVINGAREA_AVG

In [ ]:
column_name = "NONLIVINGAREA_AVG"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# APARTMENTS_MODE

In [ ]:
column_name = "APARTMENTS_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# BASEMENTAREA_MODE

In [ ]:
column_name = "BASEMENTAREA_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# YEARS_BEGINEXPLUATATION_MODE

In [ ]:
column_name = "YEARS_BEGINEXPLUATATION_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# YEARS_BUILD_MODE

In [ ]:
column_name = "YEARS_BUILD_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# COMMONAREA_MODE

In [ ]:
column_name = "COMMONAREA_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# ELEVATORS_MODE

In [ ]:
column_name = "ELEVATORS_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# ENTRANCES_MODE

In [ ]:
column_name = "ENTRANCES_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# FLOORSMAX_MODE

In [ ]:
column_name = "FLOORSMAX_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# FLOORSMIN_MODE

In [ ]:
column_name = "FLOORSMIN_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LANDAREA_MODE

In [ ]:
column_name = "LANDAREA_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LIVINGAPARTMENTS_MODE

In [ ]:
column_name = "LIVINGAPARTMENTS_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LIVINGAREA_MODE

In [ ]:
column_name = "LIVINGAREA_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NONLIVINGAPARTMENTS_MODE

In [ ]:
column_name = "NONLIVINGAPARTMENTS_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NONLIVINGAREA_MODE

In [ ]:
column_name = "NONLIVINGAREA_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# APARTMENTS_MEDI

In [ ]:
column_name = "APARTMENTS_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# BASEMENTAREA_MEDI

In [ ]:
column_name = "BASEMENTAREA_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# YEARS_BEGINEXPLUATATION_MEDI

In [ ]:
column_name = "YEARS_BEGINEXPLUATATION_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# YEARS_BUILD_MEDI

In [ ]:
column_name = "YEARS_BUILD_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# COMMONAREA_MEDI

In [ ]:
column_name = "COMMONAREA_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# ELEVATORS_MEDI

In [ ]:
column_name = "ELEVATORS_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# ENTRANCES_MEDI

In [ ]:
column_name = "ENTRANCES_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# FLOORSMAX_MEDI

In [ ]:
column_name = "FLOORSMAX_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# FLOORSMIN_MEDI

In [ ]:
column_name = "FLOORSMIN_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LANDAREA_MEDI

In [ ]:
column_name = "LANDAREA_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LIVINGAPARTMENTS_MEDI

In [ ]:
column_name = "LIVINGAPARTMENTS_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# LIVINGAREA_MEDI

In [ ]:
column_name = "LIVINGAREA_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NONLIVINGAPARTMENTS_MEDI

In [ ]:
column_name = "NONLIVINGAPARTMENTS_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NONLIVINGAREA_MEDI

In [ ]:
column_name = "NONLIVINGAREA_MEDI"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# TOTALAREA_MODE

In [ ]:
column_name = "TOTALAREA_MODE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# OBS_30_CNT_SOCIAL_CIRCLE

In [ ]:
column_name = "OBS_30_CNT_SOCIAL_CIRCLE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# DEF_30_CNT_SOCIAL_CIRCLE

In [ ]:
column_name = "DEF_30_CNT_SOCIAL_CIRCLE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.loc[df[column_name] > 2, column_name] = 2

In [ ]:
df[column_name].value_counts()

In [ ]:
df[column_name].fillna(df[column_name].mode()[0], inplace=True)
df[column_name].isna().sum()

# OBS_60_CNT_SOCIAL_CIRCLE

In [ ]:
column_name = "OBS_60_CNT_SOCIAL_CIRCLE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# DEF_60_CNT_SOCIAL_CIRCLE

In [ ]:
column_name = "DEF_60_CNT_SOCIAL_CIRCLE"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.loc[df[column_name] > 2, column_name] = 2

In [ ]:
df[column_name].value_counts()

In [ ]:
df[column_name].fillna(df[column_name].mode()[0], inplace=True)
df[column_name].isna().sum()

In [ ]:
df.shape

# AMT_REQ_CREDIT_BUREAU_HOUR

In [ ]:
column_name = "AMT_REQ_CREDIT_BUREAU_HOUR"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# AMT_REQ_CREDIT_BUREAU_DAY

In [ ]:
column_name = "AMT_REQ_CREDIT_BUREAU_DAY"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# AMT_REQ_CREDIT_BUREAU_WEEK

In [ ]:
column_name = "AMT_REQ_CREDIT_BUREAU_WEEK"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# AMT_REQ_CREDIT_BUREAU_MON

In [ ]:
column_name = "AMT_REQ_CREDIT_BUREAU_MON"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# AMT_REQ_CREDIT_BUREAU_QRT

In [ ]:
column_name = "AMT_REQ_CREDIT_BUREAU_QRT"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# AMT_REQ_CREDIT_BUREAU_YEAR

In [ ]:
column_name = "AMT_REQ_CREDIT_BUREAU_YEAR"
index_of_outliers, less_than_mean, more_than_mean, unique_values, iqr_range, lower_bound, upper_bound, column_mean_without_outliers = col_info(column_name)

In [ ]:
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

In [ ]:
categorical_columns = list(df.select_dtypes(include=["object"]).columns)
print(f"Number of categorical columns: {len(categorical_columns)}\n" )

In [ ]:
df[categorical_columns].describe()

# NAME_CONTRACT_TYPE

In [ ]:
column_name = "NAME_CONTRACT_TYPE"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
mapping_dict = {
        "Cash loans": 0,
        "Revolving loans": 1,
    }

df[column_name].replace(mapping_dict, inplace=True)
df[column_name].value_counts()

# CODE_GENDER

In [ ]:
column_name = "CODE_GENDER"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
mapping_dict = {
        "F": 0,
        "M": 1,
        "XNA": 2
    }

df[column_name].replace(mapping_dict, inplace=True)
df[column_name].value_counts()

# FLAG_OWN_CAR

In [ ]:
column_name = "FLAG_OWN_CAR"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
mapping_dict = {
        "N": 0,
        "Y": 1,
    }

df[column_name].replace(mapping_dict, inplace=True)
df[column_name].value_counts()

# FLAG_OWN_REALTY

In [ ]:
column_name = "FLAG_OWN_REALTY"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NAME_TYPE_SUITE

In [ ]:
column_name = "NAME_TYPE_SUITE"
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df[column_name].fillna("Unaccompanied", inplace=True)
df[column_name].value_counts()

In [ ]:
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NAME_INCOME_TYPE

In [ ]:
column_name = "NAME_INCOME_TYPE"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df[column_name].replace(["Student"], "Unemployed", inplace=True)
df[column_name].replace(["Maternity leave"], "Pensioner", inplace=True)
df[column_name].value_counts()

In [ ]:
mapping_dict = {
        "Working": 2,
        "Commercial associate": 2,
        "Pensioner": 1,
        "State servant": 3,
        "Unemployed": 0,
        "Businessman": 3
    }

df[column_name].replace(mapping_dict, inplace=True)
df[column_name].value_counts()

# NAME_EDUCATION_TYPE

In [ ]:
column_name = "NAME_EDUCATION_TYPE"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
mapping_dict = {
        "Secondary / secondary special": 3,
        "Higher education": 2,
        "Incomplete higher": 1,
        "Lower secondary": 0,
        "Academic degree": 4,
    }

df[column_name].replace(mapping_dict, inplace=True)
df[column_name].value_counts()

# NAME_FAMILY_STATUS

In [ ]:
column_name = "NAME_FAMILY_STATUS"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# NAME_HOUSING_TYPE

In [ ]:
column_name = "NAME_HOUSING_TYPE"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
encoder = OneHotEncoder()
encoded_df = pd.DataFrame(encoder.fit_transform(df[[column_name]]).toarray())
unique_values = df[column_name].unique()
encoded_df.columns = [column_name + "_" + str(idx) for idx in range(len(unique_values))]
df.drop(columns=[column_name], axis=1, inplace=True)
df.index = encoded_df.index
df = df.join(encoded_df)
df

In [ ]:
df.shape

# OCCUPATION_TYPE

In [ ]:
column_name = "OCCUPATION_TYPE"
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# WEEKDAY_APPR_PROCESS_START

In [ ]:
column_name = "WEEKDAY_APPR_PROCESS_START"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# ORGANIZATION_TYPE

In [ ]:
column_name = "ORGANIZATION_TYPE"
print(f"Correlation: {spearmanr(df[column_name], df['TARGET'], nan_policy='omit')[0]}")
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df[column_name].replace(["Business Entity Type 1", "Business Entity Type 2", "Business Entity Type 3"], "Business", inplace=True)
df[column_name].replace(["Transport: type 1", "Transport: type 2", "Transport: type 3", "Transport: type 4"], "Transport", inplace=True)
df[column_name].replace(["Trade: type 1", "Trade: type 2", "Trade: type 3", "Trade: type 4", "Trade: type 5", "Trade: type 6", "Trade: type 7"], "Trade", inplace=True)
df[column_name].replace(["Kindergarten", "School", "University"], "Education", inplace=True)
df[column_name].replace(["Restaurant", "Hotel"], "Dining", inplace=True)
df[column_name].replace(["Industry: type 1", "Industry: type 2", "Industry: type 3", "Industry: type 4", "Industry: type 5", "Industry: type 6", "Industry: type 7", "Industry: type 8", "Industry: type 9", "Industry: type 10", "Industry: type 11", "Industry: type 12", "Industry: type 13"], "Industry", inplace=True)
df[column_name].replace(["Insurance", "Legal Services", "Services", "Postal"], "Services", inplace=True)
df[column_name].replace(["Mobile", "Telecom"], "Mobile", inplace=True)
df[column_name].replace(["Housing", "Cleaning"], "Housing", inplace=True)
df[column_name].replace(["Realtor", "Construction"], "Realtor", inplace=True)
df[column_name].replace(["Culture", "Religion"], "Culture", inplace=True)
df[column_name].replace(["Military", "Police", "Security Ministries", "Emergency", "Security"], "Security", inplace=True)
df[column_name].value_counts()

In [ ]:
encoder = OneHotEncoder()
encoded_df = pd.DataFrame(encoder.fit_transform(df[[column_name]]).toarray())
unique_values = df[column_name].unique()
encoded_df.columns = [column_name + "_" + str(idx) for idx in range(len(unique_values))]
df.drop(columns=[column_name], axis=1, inplace=True)
df.index = encoded_df.index
df = df.join(encoded_df)
df

# FONDKAPREMONT_MODE

In [ ]:
column_name = "FONDKAPREMONT_MODE"
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# HOUSETYPE_MODE

In [ ]:
column_name = "HOUSETYPE_MODE"
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# WALLSMATERIAL_MODE

In [ ]:
column_name = "WALLSMATERIAL_MODE"
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

# EMERGENCYSTATE_MODE

In [ ]:
column_name = "EMERGENCYSTATE_MODE"
print(f"Missing value(s): {df[column_name].isna().sum()}")
df[column_name].value_counts()

In [ ]:
df.drop(columns=[column_name], inplace=True)
df.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
new_df = df.copy()
new_df

In [ ]:
new_df_dropped = new_df.drop(columns=["SK_ID_CURR"], axis=1)
new_df_dropped

In [ ]:
num_columns_new_df_dropped = list(new_df_dropped.select_dtypes(exclude=["object"]).columns)
target_column = new_df_dropped[["TARGET"]]
to_scale = new_df_dropped[num_columns_new_df_dropped]
to_scale = to_scale.drop(["TARGET"], axis=1)

In [ ]:
to_scale

In [ ]:
scaler = StandardScaler()
scaler.fit(to_scale)

In [ ]:
scaled = scaler.transform(to_scale)
scaled_df = pd.DataFrame(scaled, columns=to_scale.columns)

In [ ]:
scaled_df

In [ ]:
scaled_complete_df = pd.concat([scaled_df, target_column], axis=1)
scaled_complete_df

In [ ]:
predictors = scaled_complete_df.iloc[:, :-1].values
labels = scaled_complete_df.iloc[:, -1]

In [ ]:
predictors.shape

In [ ]:
labels.shape

In [ ]:
clf = LogisticRegression(
    random_state=42,
    max_iter=3000,
    # solver="saga",
    # penalty="elasticnet",
    # l1_ratio=0.5,
    n_jobs=-1,
)

In [ ]:
stratified_splits = StratifiedShuffleSplit(
            n_splits=5,
            test_size=0.25,
            random_state=42,
        )

In [ ]:
iter = 0
for train_indices, val_indices in stratified_splits.split(predictors,
                                                          labels):
    iter += 1
    print(f"training model - iteration {iter}")
    X_train, X_val = predictors[train_indices], predictors[val_indices]
    y_train, y_val = labels.loc[train_indices], labels.loc[val_indices]
    clf.fit(X_train, y_train)
    print(f"complete training model - iteration {iter}")
    y_predict = clf.predict(X_val)
    print(f"calculating classification scores")
    f1_macro = f1_score(y_val, y_predict, average="macro")
    print(f"f1 score (macro) score is: {f1_macro}")